<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [11]</a>'.</span>

In [23]:
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder
import torch
from create_database_functions import *
import json

In [24]:
file_path = "C:/Users/Iraklis Kolokas/Desktop/Thesis/Features"

Get features

In [25]:
all_data = get_files(file_path)

Cosine Similarity

Averages between each session and for features from the same acoustic stimuli

In [26]:
av_per_user, each_freq_av = get_averages(all_data)

Neural Network

In [27]:
#Check if the system has a GPU
#cuda = torch.cuda.is_available()
#device = "cuda" if cuda else "cpu"
#if cuda:
    #torch.backends.cudnn.benchmark = True 

In [28]:
features_1, stimuli_1, user_ids_1 = make_lists(all_data, 1)
features_2, stimuli_2, user_ids_2 = make_lists(all_data, 2)

In [29]:
all_data.values()

dict_values([{'features': array([[[ 9.48685397e-09,  8.19448047e-08, -7.15742601e-07, ...,
          9.30605950e-07,  1.55375960e-06,  5.64960942e-07],
        [-8.29459168e-09, -4.95592531e-07, -5.91798312e-07, ...,
         -7.89372905e-07, -1.75770268e-07,  2.08525776e-07],
        [ 1.49225059e-07, -4.97379915e-09, -1.08939105e-07, ...,
         -5.17905619e-07, -9.54167035e-07, -1.18978380e-06],
        ...,
        [-5.15345789e-07, -1.28823388e-07,  5.65957919e-07, ...,
          5.87057002e-07, -4.13326120e-07,  2.04015095e-07],
        [-8.20010428e-07, -9.60640492e-08,  9.00830059e-07, ...,
         -1.00937189e-06, -1.21425074e-07, -1.53665098e-08],
        [-1.20604255e-07,  8.25811810e-08, -3.69841189e-07, ...,
          1.73073317e-06,  4.47387862e-07, -1.13322358e-07]],

       [[-2.88660635e-06, -2.62868525e-06, -3.34061588e-06, ...,
         -1.93290134e-06, -2.18392620e-06, -4.53692905e-07],
        [ 1.88356307e-06,  3.92276377e-07, -3.33972281e-07, ...,
         -1.

In [36]:
print(np.array(features_1).shape)
print(np.array(features_2).shape)

(38, 15, 24, 89)
(38, 15, 20, 64)


In [31]:
x_1, x_2, y_1, y_2 = make_databases(features_1, features_2, user_ids_1, user_ids_2)

In [32]:
#Remap labels to fit the training
unique_labels = np.unique(y_1)
label2idx = {label: idx for idx, label in enumerate(unique_labels)}  # Original to index
idx2label = {int(k): int(v) for k, v in label2idx.items()}

y_1_remapped = np.array([label2idx[label.item()] for label in y_1])
y_2_remapped = np.array([label2idx[label.item()] for label in y_2])
y_1_remapped = torch.tensor(y_1_remapped,dtype = torch.long)
y_2_remapped = torch.tensor(y_2_remapped,dtype = torch.long)

#Save the mapping for later use
with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/label_mapping.json", "w") as f:
    json.dump(idx2label, f)

In [33]:
dataset_1 = TensorDataset(x_1, y_1_remapped)
loader_1 = DataLoader(dataset_1, batch_size = 32, shuffle = True)

dataset_2 = TensorDataset(x_2, y_2_remapped)
loader_2 = DataLoader(dataset_2, batch_size = 32, shuffle = True)

In [35]:
print(x_1.shape)
print(x_2.shape)
#print(y_2_remapped.shape)
print(y_1.shape)

torch.Size([570, 24, 89])
torch.Size([570, 20, 64])
torch.Size([570])


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [ ]:
#Train the feature-only model
model1 = UserClassifier(x_1.shape[1], x_1.shape[3], len(torch.unique(y_1)))
trained_model1 = train_classifier(model1, loader_1, n_epochs = 10)#, device = device)

model2 = UserClassifier(x_2.shape[1], x_2.shape[3], len(torch.unique(y_2)))
trained_model2 = train_classifier(model2, loader_2, n_epochs = 10)#, device = device)

In [ ]:
#Add the stimuli labels to each feature with OneHotEncoder
stimuli_1 = np.hstack(stimuli_1)
stimuli_2 = np.hstack(stimuli_2)

stimuli_1 = stimuli_1.reshape(-1,1)
stimuli_2 = stimuli_2.reshape(-1,1)

encoder = OneHotEncoder(sparse_output = False)
stimuli_1 = encoder.fit_transform(stimuli_1)
stimuli_2 = encoder.fit_transform(stimuli_2)

#Add the encoded stimuli labels to the corresponding feature
features_1 = np.hstack([features_1, stimuli_1])
features_2 = np.hstack([features_2, stimuli_2])

In [ ]:
x_1, x_2, y_1, y_2 = make_databases(features_1, features_2, user_ids_1, user_ids_2)

In [ ]:
y_1_remapped = np.array([label2idx[label.item()] for label in y_1])
y_2_remapped = np.array([label2idx[label.item()] for label in y_2])
dataset_1 = TensorDataset(x_1, y_1_remapped)
loader_1 = DataLoader(dataset_1, batch_size = 32, shuffle = True)

dataset_2 = TensorDataset(x_2, y_2_remapped)
loader_2 = DataLoader(dataset_2, batch_size = 32, shuffle = True)

In [ ]:
#Train the feature-label model
model3 = EEGUserClassifier(x_1.shape[1], x_1.shape[3], len(torch.unique(y_1)))
trained_model3 = train_classifier(model3, loader_1, n_epochs = 10)

model4 = EEGUserClassifier(x_2.shape[1], x_1.shape[3], len(torch.unique(y_2)))
trained_model4 = train_classifier(model4, loader_2, n_epochs = 10)

Save all the models

In [ ]:
torch.save(trained_model1.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_1.pth")
torch.save(trained_model2.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_2.pth")
torch.save(trained_model3.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_w_labels_1.pth")
torch.save(trained_model4.state_dict(), "C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/user_classifier_w_labels_2.pth")

Save averages

In [ ]:
#Save as pickle file to preserve dictionary structure
with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/each_freq_av.pkl" "wb") as f:
    pickle.dump(each_freq_av, f)
    
with open("C:/Users/Iraklis Kolokas/Desktop/Thesis/Databases/av_per_user.pkl" "wb") as f:
    pickle.dump(av_per_user, f)